In [1]:
from pyspark.sql import SparkSession
from malaya.tokenizer import SentenceTokenizer
from pyspark.sql.functions import array, col, concat_ws
from pyspark.sql import Row
from itertools import chain
import re

class DE_Transformation:
    def __init__(self):
        self.spark = None


# Example list of strings

    # Function to remove special characters
    def remove_special_characters(text):
        return re.sub(r'[^A-Za-z0-9\s]', '', text)


    
    def readRawData(self):
        self.spark = spark = SparkSession\
                .builder\
                .appName("DE_malay_news")\
                .getOrCreate()

        df = self.spark.read.parquet("DE-prj/RawData").limit(10)
        #drop duplicated
        df = df.drop_duplicates(['url']).select(['title','articleBrief','contents'])
        
        #Merge DF
        df = df.withColumn("Flattened_Contents", concat_ws(" ", col("contents")))
        df = df.select(['title','articleBrief','Flattened_Contents'])
        df.select(['Flattened_Contents']).show(truncate=False)
        merged_df = df.withColumn("Merged", array(*[col(c) for c in df.columns])).select(['Merged'])

        return merged_df




    
    
    def sentenceSegmentationWithSpark(self, df):
        self.spark = spark = SparkSession\
                .builder\
                .appName("DE_malay_news")\
                .getOrCreate()
        
        lists = df.collect()
        # Flatten the 'contents' column
        flattened_list = list(chain.from_iterable(row.Merged for row in lists))
        print(len(flattened_list))
        s_tokenizer = SentenceTokenizer()
        rddInput = self.spark.sparkContext.parallelize(flattened_list)
        # removedSpecialChar = rddInput.map(DE_Transformation.remove_special_characters)
        # df = removedSpecialChar.collect()
        result = []
        for strr in flattened_list:
            oneResult = s_tokenizer.tokenize(strr)
            if len(oneResult) > 0:
                result.append(oneResult)
        # rddInput = self.spark.sparkContext.parallelize(df)
        result = rddInput.map(DE_Transformation().sentenceSegmentationProcess).take(30)
        

        return result

    def sentenceSegmentationProcess(self,strList):
        s_tokenizer = SentenceTokenizer()
        # result = []
        # for ele in strList:
        #     oneResult = s_tokenizer.tokenize(strList)
        #     if len(oneResultreturn oneResult0:
        #         result.append(oneResult)

        # return result
        oneResult = s_tokenizer.tokenize(strList)
        if len(oneResult) > 0:
            return oneResult
        else: 
            return ["",'']

    def sentenceSegmentation(self, df):
        lists = df.collect()
        # Flatten the 'contents' column
        flattened_list = list(chain.from_iterable(row.Merged for row in lists))
        s_tokenizer = SentenceTokenizer()
        result = []
        for str in flattened_list:
            result.append(s_tokenizer.tokenize(str))


        return result 
df = DE_Transformation().readRawData()
results = DE_Transformation().sentenceSegmentationWithSpark(df)


#print(result.collect())

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/12 02:14:27 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/12 02:14:28 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/12 02:14:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

[Stage 7:>                                                          (0 + 1) / 1]/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
[Stage 8:==============>                                            (1 + 3) / 4]/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/ma

Py4JError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob

In [14]:
for ele in results:
    print(ele)

['Peraga kemewahan, tapi pinjaman MARA tak bayar - Asyraf Wajdi.']
['SEREMBAN: "Jangan pula ada masalah atau cari masalah untuk sengaja tak nak bayar pinjaman tetapi dalam masa sama memperagakan kemewahan, jadi ini saya rasa tidak adil bagi semua pihak," kata Pengerusi Majlis Amanah Rakyat (MARA), Datuk Asyraf Wajdi Dusuki.', 'Beliau membidas tindakan sesetengah peminjam institusi itu yang dilihat sengaja tidak mahu membayar pinjaman tetapi memperagakan kemewahan.', 'Asyraf Wajdi berkata, Mara tidak pernah memilih bulu sebaliknya mengambil pendekatan berunding dan sentiasa bersikap ihsan kepada sesiapa yang bermasalah dalam melangsaikan hutang pinjaman.', '"Saya tiada kuasa untuk melupuskan apa jua pinjaman.', 'Kalau ada masalah datang jumpa (pihak MARA)," katanya kepada pemberita di sini, semalam.', 'Asyraf Wajdi memberi reaksi itu selepas pendedahan media hari ini mengenai rumah pelakon terkenal, Rozita Che Wan, 51, atau lebih dikenali sebagai Che Tadiserbu pegawai MARA susulan kegag